### Anime recommender using cosine similarities on its properties like actors,directors and genres 
https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

In [8]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
pd.options.mode.chained_assignment = None  # default='warn'
df2=pd.read_csv('dataset/credits.csv')
df1= pd.read_csv("dataset/titles.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Group by 'title' and aggregate 'artist' using a lambda function to join multiple artists
grouped_df = df2.groupby('id')['name'].agg(lambda x: ', '.join(x)).reset_index()

# Reset index to get rid of hierarchical index and create a new DataFrame
new_df = pd.DataFrame(grouped_df)
new_df.head()

,id,name
0,tm176185,"Romi Park, Rie Kugimiya, Toru Okawa, Megumi To..."
1,ts10145,"Minami Takayama, Kappei Yamaguchi, Wakana Yama..."
2,ts102805,"Rie Takahashi, Nobunaga Shimazaki, Yu Kobayash..."
3,ts10585,"Takehito Koyasu, Daisuke Ono, Yuu Asakawa, Tet..."
4,ts10963,"Asako Dodo, Kousuke Okano, Nobuo Tobita, Akiko..."


In [10]:

df=df1.merge(new_df,on='id')
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,name
0,ts21930,Lupin the Third,SHOW,Arsene Lupin III is the grandson of the master...,1971,TV-14,23,"['scifi', 'animation', 'action', 'comedy', 'cr...",['JP'],6.0,tt0159175,7.9,2116.0,45.829,8.0,"Kanichi Kurita, Daisuke Namikawa, Miyuki Sawas..."
1,ts25177,Mobile Suit Gundam,SHOW,Mobile Suit Gundam is a televised anime series...,1979,TV-Y7,25,"['scifi', 'war', 'animation', 'action', 'drama...",['JP'],1.0,tt0159172,7.9,2281.0,41.250,8.1,"Toru Furuya, Shuuichi Ikeda, Hirotaka Suzuoki,..."
2,ts35087,Galaxy Express 999,SHOW,Galaxy Express 999 is a manga written and draw...,1978,TV-PG,24,"['scifi', 'action', 'drama', 'animation']",['JP'],1.0,tt0185066,7.8,459.0,5.064,8.3,"Masako Nozawa, Masako Ikeda, Hitoshi Takagi, K..."
3,ts2846,Chōdenji Machine Voltes V,SHOW,Chōdenji Machine Voltes V is a Japanese anime ...,1977,TV-PG,24,"['drama', 'action', 'scifi', 'animation']",['JP'],1.0,tt0159855,8.1,207.0,12.625,7.3,"Kazuyuki Sogabe, Seizō Katō, Noriko Ohara, Hir..."
4,ts32935,Space Pirate Captain Harlock,SHOW,"Follows Captain Harlock, an outcast turned spa...",1978,TV-PG,25,"['action', 'scifi', 'animation', 'drama', 'fan...",['JP'],1.0,tt0182646,7.8,1159.0,16.793,6.7,"Makio Inoue, Akira Kamiya, Noriko Ohara, Chiyo..."


In [11]:
df=df.dropna()


In [12]:
df['Key_words'] = " "

for index, row in df.iterrows():
    description = row['description']
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(description)
    

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    df.at[index, 'Key_words'] = list((key_words_dict_scores.keys()))

In [13]:
df.reset_index(drop=True, inplace=True) #now that index is good i can train again

In [14]:
df["bag_of_words"]=""

genre = ['genres']
keyword= ['Key_words']
name=['name']
titles=['title','age_certification']
prod = ['production_countries']

for index, row in df.iterrows():
    words = ''
    for col in titles:
        words = words + ''.join(row[col])+' '
    for col in genre:
        row[col] = row[col].replace('[', '').replace(']', '').replace('\'', '')
        row[col] = row[col].replace(',', ' ')
        words = words + ''.join(row[col])+' '
    for col in name:
        row[col] = row[col].replace(',', ' ')
        words = words + ' '.join(row[col])+' '

    # for col in keyword:
    #     words = words + ' '.join(row[col])+ ' '
    # for col in prod:    
    #     row[col] = row[col].replace('[', '').replace(']', '').replace('\'', '')
    # for col in scores:
    #     words = words + str(int(row[col]))
    df.at[index,'bag_of_words'] = words.lower()
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,name,Key_words,bag_of_words
0,ts21930,Lupin the Third,SHOW,Arsene Lupin III is the grandson of the master...,1971,TV-14,23,"['scifi', 'animation', 'action', 'comedy', 'cr...",['JP'],6.0,tt0159175,7.9,2116.0,45.829,8.0,"Kanichi Kurita, Daisuke Namikawa, Miyuki Sawas...","[arsene, lupin, iii, grandson, master, thief, ...",lupin the third tv-14 scifi animation action...
1,ts25177,Mobile Suit Gundam,SHOW,Mobile Suit Gundam is a televised anime series...,1979,TV-Y7,25,"['scifi', 'war', 'animation', 'action', 'drama...",['JP'],1.0,tt0159172,7.9,2281.0,41.250,8.1,"Toru Furuya, Shuuichi Ikeda, Hirotaka Suzuoki,...","[mobile, suit, gundam, televised, anime, serie...",mobile suit gundam tv-y7 scifi war animation...
2,ts35087,Galaxy Express 999,SHOW,Galaxy Express 999 is a manga written and draw...,1978,TV-PG,24,"['scifi', 'action', 'drama', 'animation']",['JP'],1.0,tt0185066,7.8,459.0,5.064,8.3,"Masako Nozawa, Masako Ikeda, Hitoshi Takagi, K...","[galaxy, express, 999, manga, written, drawn, ...",galaxy express 999 tv-pg scifi action drama ...
3,ts2846,Chōdenji Machine Voltes V,SHOW,Chōdenji Machine Voltes V is a Japanese anime ...,1977,TV-PG,24,"['drama', 'action', 'scifi', 'animation']",['JP'],1.0,tt0159855,8.1,207.0,12.625,7.3,"Kazuyuki Sogabe, Seizō Katō, Noriko Ohara, Hir...","[chōdenji, machine, voltes, v, japanese, anime...",chōdenji machine voltes v tv-pg drama action ...
4,ts32935,Space Pirate Captain Harlock,SHOW,"Follows Captain Harlock, an outcast turned spa...",1978,TV-PG,25,"['action', 'scifi', 'animation', 'drama', 'fan...",['JP'],1.0,tt0182646,7.8,1159.0,16.793,6.7,"Makio Inoue, Akira Kamiya, Noriko Ohara, Chiyo...","[follows, captain, harlock, outcast, turned, s...",space pirate captain harlock tv-pg action sci...


In [15]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

indices = pd.Series(df['title'])
print(indices[:5]) 
#index to access to get recommendation
cosine_sim = cosine_similarity(count_matrix, count_matrix)

0                 Lupin the Third
1              Mobile Suit Gundam
2              Galaxy Express 999
3       Chōdenji Machine Voltes V
4    Space Pirate Captain Harlock
Name: title, dtype: object


In [16]:

def recommendation(Title, cosine_sim = cosine_sim):
    
    recommended_anime = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == Title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_5_indexes = list(score_series.iloc[1:6].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_5_indexes:
            recommended_anime.append(list(df.title)[i])
        
    return recommended_anime

In [17]:
recommendation('Your Lie in April')

['Gravitation', 'White Album', 'Orange', 'Glass Mask', 'Hyouka']

In [18]:
df[df.title=='Domestic Girlfriend']
idx = indices[indices == 'Domestic Girlfriend'].index[0]
print(idx)

# creating a Series with the similarity scores in descending order
score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)


497


In [19]:
df[df.title=='Dragon Ball']
idx = indices[indices == 'Dragon Ball'].index[0]
print(idx)

# creating a Series with the similarity scores in descending order
score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
print(score_series)

6
6      1.000000
263    0.877058
20     0.858116
152    0.782624
510    0.782624
         ...   
237    0.223607
421    0.223607
374    0.223607
189    0.200000
85     0.190693
Length: 609, dtype: float64


In [20]:
df[df.title=='Domestic Girlfriend']

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,name,Key_words,bag_of_words
497,ts84640,Domestic Girlfriend,SHOW,Natsuo is a high school boy who is experiencin...,2019,TV-MA,23,"['drama', 'romance', 'animation']",['JP'],1.0,tt9817980,6.8,2950.0,33.047,8.5,"Maaya Uchida, Yoko Hikasa, Taku Yashiro, Takuy...","[natsuo, high, school, boy, experiencing, crus...",domestic girlfriend tv-ma drama romance anim...


In [21]:
recommendation('Domestic Girlfriend')

['Kaiba',
 'My First Girlfriend is a Gal',
 'Junjou Romantica',
 'White Album 2',
 'Fuuka']